In [1]:
import pymongo
import pprint
from pymongo import MongoClient
import seaborn as sns
import matplotlib.pyplot as plt
from flask import Flask, render_template

In [2]:
#Establish a connection to your MongoDB server
# client = MongoClient('130.238.29.169', 27017)
from pymongo import MongoClient
# MongoDB connection details
host = '130.238.29.169'
port = 27017
username = 'root'
password = 'example'
# Create the MongoDB URI with authentication
uri = f"mongodb://{username}:{password}@{host}:{port}"
# Establish a connection to your MongoDB server with authentication
client = MongoClient(uri)
# Now you can work with the authenticated connection
# For example, you can access collections in the database
db = client['githubdata']
collection = db['repositories_v1']

In [3]:
#Group the documents by language and count the projects
pipeline1 = [
    {'$group': {'_id': '$language', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}},
    {'$limit': 10}
]
#Perform the aggregation query
result = collection.aggregate(pipeline1)
#Retrieve and print the top 10 programming languages
languages = []
counts = []
for doc in result:
    languages.append(doc['_id'])
    counts.append(doc['count'])

In [4]:
top_projects = collection.find(
    {"updated_commits": True},
    {"full_name": 1, "commits": 1}
).sort("commits", -1).limit(10)
# get the top 10 projects
projects = []
proj_counts = []
for doc in top_projects:
    projects.append(doc['full_name'])
    proj_counts.append(doc['commits'])

In [5]:
results = collection.aggregate([
    {
        '$match': {
            'updated_unit_tests': True
        }
    },
    {
        '$group': {
            '_id': '$language',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    },
    {
        '$limit': 10
    }
])
# Print the top 10 programming languages
td_languages = []
td_count = []
for result in results:
    td_languages.append(result["_id"])
    td_count.append(result["count"])

In [6]:
# Query the database
pipeline = [
    {
        '$match': {
            "updated_unit_tests": True,
            "updated_cicd": True
        }
    },
    {
        '$group': {
            '_id': '$language',
            'count': {'$sum': 1}
        }
    },
    {
        '$sort': {'count': -1}
    },
    {
        '$limit': 10
    }
]
results = collection.aggregate(pipeline)
#top 10 programming languages that follow test-driven development and DevOps approach
de_languages = []
de_count = []
for result in results:
    de_languages.append(result['_id'])
    de_count.append(result['count'])

In [7]:
app = Flask(__name__)

@app.route('/')
def home():
    # Data for the 1st graph 
    lang = languages
    project_counts = counts
    # Data for the 2nd graph 
    p_name = projects
    p_count = proj_counts
    # Data for the 3rd graph
    td_lang = td_languages
    t_count = td_count
    # Data for the 4th graph
    de_lang = de_languages
    d_count = de_count
    return render_template('index.html', title='Results of Analysis', lang=languages, project_counts=counts, p_name = projects, p_count = proj_counts, td_lang = td_languages, t_count = td_count, de_lang = de_languages, d_count = de_count)

In [9]:
if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.132:5000
Press CTRL+C to quit
130.243.231.115 - - [27/May/2023 12:07:47] "GET / HTTP/1.1" 200 -
